# Práctica NoSQL y Elastic

Conexión kibana: http://127.0.0.1:5601

## Datasets

### Las obras completas de Wiliam Sakespeare

Este conjunto de datos contiene los diálogos de todas la obras de Wiliam Sakespeare. Cada línea identifica una frase de un diálogo y al personaje al que corresponde el texto. Ejemplo:

- line_id: Identificador único de la línea de diálogo. Tipo entero.
- play_name: Nombre de la obra.
- speach_number: Número del diálogo.
- line_number: Número de línea del diálogo en la obra.
- speaker: Personaje que dice el téxto.
- text_entry: Texto dicho por el personaje en el diálogo.

Por la forma en la que está estructurada la información en este conjunto de datos y los posibles casos de uso aplicables (número de líneas de cada personaje, palabras más utilizadas, palabra más usada por un personaje, etc.) lo recomendable sería usar Elasticsearch, ya que se trata de un motor de búsqueda completo basado en Lucene.

Elasticsearch es capaz de tokenizar el texto para después almacenarlo en un índice invertido, lo cual permite identificar los documentos en los que aparece cada palabra.

### Calidad del Aire de la ciudad de Madrid

Este conjunto de datos hace referencia a la calidad del aire de la ciudad de Madrid y recoge las mediciones horarias de distintas métricas en diferentes estaciones base.

- PROVINCIA: Código de provincia (no aporta información ya que todos son de Madrid)
- MUNICIPIO: Código de municipio (tampoco aportaría mucho)
- ESTACION: Código de estación
- MAGNITUD: Código de la magnitud que se está midiendo
- PUNTO_MUESTREO: Código de muestre
- ANO: Año de medida
- MES: Mes de medida
- DIA: Día de medida
- H01: Valor de la magnitud medida a las 01:00h
- V01: Código de verificación de la medida
- H02: Valor de la magnitud medida a las 02:00h
- V02: Código de verificación de la medida
- ...
- H24: Valor de la magnitud medida a las 24:00h
- V24: Código de verificación de la medida

Por la forma en la que está estructurada la información en este conjunto de datos y los posibles casos de uso aplicables (evolución horaria de una o varias magnitudes, comparación de magnitudes entre estaciones base, etc.), lo recomendable sería usar Cassandra ya que es una base de datos distribuida altamente escalable y de alto rendimiento que es capaz de manejar grandes volúmenes de datos en tiempo real.

### City Bike

Este conjunto de datos contiene información sobre todos los trayectos realizados por los usuarios del sistema de bicicletas públicas de la ciudad de Nueva York.

Cada entrada del dataset tiene los siguientes campos:

- Trip Duration: Duración del viaje en segundos.
- Start Time and Date: Marca de tiempo con el inicio del trayecto.
- Stop Time and Date: Marca de tiempo con el final del trayecto.
- Start Station Name: Nombre de la estación de inicio.
- End Station Name: Nombre de la estación de destino
- Station ID: Identificador único de la estación de inicio.
- Station Lat/Long: Latitud y longitud de la estación de inicio.
- Bike ID: Identificador único de la bicicleta utilizada para el trayecto.
- User Type: Tipo de usuario que ha realizado el trayecto.
  - Customer:
    - 24-hour pass
    - 3-day pass user
  - Subscriber = Annual Member 
- Gender (Zero=unknown; 1=male; 2=female): Género del usuario que ha realizado el trayecto.
- Year of Birth: Año de nacimiento del usuario que ha realizado el trayecto.

Por la forma en la que está estructurada la información en este conjunto de datos y los posibles casos de uso aplicables (estación más usada por usuarios subscritos o usuarios temporales, usuarios que han cambiado de tipo de subscripción, tipo de trayecto más habitual entre tipos de usurios, trayectos más frecuentes, horas puntas / horas valle, etc.) , lo recomendable sería usar Neo4j ya que una de sus ventajas principales es su capacidad para manejar relaciones complejas entre datos de manera eficiente.

### Log Files

Este conjunto de datos contiene ifnormación de ficheros de log de un servido web.

Cada entrada tiene los siguientes campos:
    
- @timestamp: Marca de tiempo de la petición web registrada en el log.
- ip: IP de origen de la petición.
- extension: Tipo de recurso pedido.
- response: Código de respuesta http del servirdor a la petición.
- geo: Información geográfica asociada a la IP de origen de la petición.
- @tags: Etiquetas asociadas a la petición.
- utc_time: Marca de tiempo de la petición en UTC.
- referer: URL del host desde donde se hizo la petición.
- agent: User Agent del cliente que realizó la petición.
- clientip: IP del cliente que realizó la petición.
- bytes: Tamaño de la respuesta en bytes.
- host: URL del host de la aplicación.
- url: URL de la petición realizada al servidor.
- @message: Mendaje del log.
- headings: Cabecera de la página pedida.
- links: Enlaces que contiene la página pedida.
- relatedContent: Contenido relacionado con la petición.
- machine: Información sobre el dispositivo del cliente que realizó la petición.
- @version: Versión del mensaje.


Por la forma en la que está estructurada la información en este conjunto de datos y los posibles casos de uso aplicables (némero de peticiones en un espacio de tiempo, tipo de recurso solicituda, peticiones dentro de una zona geográfica, tamaños de las respuetas, número de respuestas con error del servidor, etc.) lo recomendable sería usar Elasticsearch.

## Descripcion del caso de uso

Se va a crear un caso de uso basado en el dataset "Log Files". Como se ha comentado anteriormente, el uso de elasticsearch se considera lo más apropiado debido a las características de los datos que se van a almacenar.

Se ha optado por usar Elasticsearch porque es de aplicación en mi ámbito laboral. Se descarta el uso de Logstash por ser una herramienta que no usaré en mi puesto, a pesar de que lo más adecuado sería su uso.

**El caso de uso ideado consiste en crear un proceso que diariamente consulte el enlace de descarga donde se va a encontrar un único fichero correspondiente al log del servidor web del día anterior. Una vez descargado el fichero de log se creará un indice en elasticsearch correspondiente al día actual con los logs del día anterior.**

Se diseña un template para los íncides de modo que las búsquedas pueden realizarse sobre los índices de todos los días.

Además, se creará la siguiente política de gestión de ciclo de vida:
- Hot: Se utiliza para alojar los datos de cada día. Se pueden añadir, borrar, modificar y consultar documentos.
- Warm: No se pueden modificar pero estan disponible durante unos días para consultar el funcionamiento del servidor web.
- Delete: Los índices se borran ya que los documentos se consideran que no tienen validez o no son útiles.

Para probar la política de ciclo de vida de los índices y de paso, reducir el tiempo para la carga de documentos en elasticsearch, se ha creado una varible **test** que permite recolectar un subconjunto de registros del fichero de log facilitado para la práctica.

**IMPORNTATE: Si test = False, se cargarán todos los documentos en el índice del día.**

In [ ]:
import os
from IPython.display import display, Image
file_imagen = os.path.join(os.getcwd(), 'imagen.png')
display(Image(filename=file_imagen))

## Diagrama del modelo de datos

Tras analizar algunas líneas del dataset seleccionado, se define la siguiente plantilla de modelo de datos:
- Incluido en el fichero index_template_curl.txt

In [ ]:
! curl -X PUT "http://elasticsearch:9200/_index_template/index-web-log"  -H 'Content-Type: application/json' -d' \
{ \
  "index_patterns": ["index-web-log-*"], \
  "template": { \
    "settings":{ \
      "number_of_shards":3, \
      "number_of_replicas":2, \
      "index.lifecycle.name": "web-log-metrics-policy", \
      "index.lifecycle.rollover_alias": "index-web-log" \
    }, \
    "aliases":{ \
      "all_orders":{} \
    }, \
    "mappings": { \
      "properties": { \
        "@timestamp": { \
            "type": "date" \
            }, \
        "ip": { \
            "type": "ip" \
            }, \
        "extension": { \
            "type": "keyword" \
            }, \
        "response": { \
            "type": "keyword" \
            }, \
        "geo": { \
            "properties": { \
                "coordinates": { \
                    "type": "geo_point" \
                    }, \
                "src": { \
                    "type": "keyword" \
                    }, \
                "dest": { \
                    "type": "keyword" \
                    }, \
                "srcdest": { \
                    "type": "text" \
                    } \
                } \
            }, \
        "@tags": { \
            "type": "text" \
            }, \
        "utc_time": { \
            "type": "date" \
            }, \
        "referer": { \
            "type": "text" \
            }, \
        "agent": { \
            "type": "text" \
            }, \
        "clientip": { \
            "type": "ip" \
            }, \
        "bytes": { \
            "type": "integer" \
            }, \
        "host": { \
            "type": "text" \
            }, \
        "request": { \
            "type": "text" \
            }, \
        "url": { \
            "type": "text" \
            }, \
        "@message": { \
            "type": "text" \
            }, \
        "spaces": { \
            "type": "text" \
            }, \
        "xss": { \
            "type": "text" \
            }, \
        "headings": { \
            "type": "text" \
            }, \
        "links": { \
            "type": "text" \
            }, \
        "relatedContent": { \
            "properties": { \
                "url": { \
                    "type": "text" \
                    }, \
                "og:type": { \
                    "type": "keyword" \
                    }, \
                "og:title": { \
                    "type": "text" \
                }, \
                "og:description": { \
                    "type": "text" \
                    }, \
                "og:url": { \
                    "type": "text" \
                    }, \
                "article:published_time": { \
                    "type": "date" \
                    }, \
                "article:modified_time": { \
                    "type": "date" \
                    }, \
                "article:section": { \
                    "type": "keyword" \
                    }, \
                "article:tag": { \
                    "type": "keyword" \
                    }, \
                "og:image": { \
                    "type": "text" \
                    }, \
                "og:image:height": { \
                    "type": "integer" \
                    }, \
                "og:image:width": { \
                    "type": "integer" \
                    }, \
                "og:site_name": { \
                    "type": "keyword" \
                    }, \
                "twitter:title": { \
                    "type": "text" \
                    }, \
                "twitter:description": { \
                    "type": "text" \
                    }, \
                "twitter:card": { \
                    "type": "keyword" \
                    }, \
                "twitter:image": { \
                    "type": "text" \
                    }, \
                "twitter:site": { \
                    "type": "keyword" \
                    } \
                } \
            }, \
        "machine": { \
            "properties": { \
                "os": { \
                    "type": "keyword" \
                    }, \
                "ram": { \
                    "type": "long" \
                    } \
                } \
            }, \
        "@version": { \
            "type": "keyword" \
            }, \
        "memory": { \
            "type": "long" \
            }, \
        "phpmemory": { \
            "type": "long" \
            } \
      } \
    } \
  } \
}'

## Política de gestion de ciclo de vida de índices (ILM)

Tal y como se ha comentados antes, se establece un ciclo de vida de los índices:
- Hot: El índice se rotará cuando tenga más de 1 día o más de 1Gb.
- Warm: Los índices de más de 1 días y menos de 14 días.
- Delete: Los índices de más de 14 días se borrarán.

Tras consultar la siguiente página

https://www.elastic.co/guide/en/elasticsearch/reference/current/size-your-shards.htmlhttps://www.elastic.co/guide/en/elasticsearch/reference/current/size-your-shards.html

se indica que la opción "max_age" está en proceso de desuso, por lo que se opta por usar la recomendación de usar "max_primary_shard_size"

_If you use ILM and your retention policy allows it, avoid using a max_age threshold for the rollover action. Instead, use max_primary_shard_size to avoid creating empty indices or many small shards._

In [ ]:
! curl -X PUT "http://elasticsearch:9200/_ilm/policy/web-log-metrics-policy" -H 'Content-Type: application/json' -d' \
{ \
    "policy": { \
        "phases": { \
            "hot": { \
                "actions": { \
                    "rollover": { \
                        "max_age": "1d", \
                        "max_primary_shard_size": "1gb" \
                    } \
                } \
            }, \
            "warm": { \
                "min_age": "1d", \
                "actions": {} \
            }, \
            "delete": { \
                "min_age": "14d", \
                "actions": {} \
            } \
        } \
    } \
}'


Se crea el _bootstrap index_ que es el índice inicial y en el que se empezarán a añadir los datos. Será el primer índice del nivel Hot.

**IMPORTANTE: MODIFICAR INDICADO LA FECHA (YYYYMMDD) EN LA QUE SE VA A EJECUTAR.** Al elaborar esta memoria se fijó como primer índice el correspondiente al día 21 de febrero de 2024 (20220221)

In [ ]:
! curl -X PUT "http://elasticsearch:9200/index-web-log-20240221" -H 'Content-Type: application/json' -d' \
{ \
    "aliases": { \
        "index-web-log":{ \
            "is_write_index": true \
        } \
    } \
}'

## Ingesta de datos

### Prerequisitos

En esta práctica se va a usar el cliente Python para Elasticsearh:
https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/installation.html

Con el fin de evitar posibles problemas, se recomenda instalar y/o actualizar algunas liberías.

In [ ]:
!pip install requests==2.27.1

In [ ]:
!pip install elasticsearch

### Importación de liberías

In [ ]:
import os
import json
import random
import pandas as pd

from datetime import datetime, timedelta
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

### Conexion con Elasticsearch

In [ ]:
elastic_con = Elasticsearch('http://elasticsearch:9200')

### Creación del índice diario

Para enviar documentos a Elasticsearch mediante el cliente de Python se requiere en primer lugar crear el índice donde se van a guardar. Para ello se necesita especificar el nombre del índice y en el cuerpo de la petición, los _settings_ y el _mapping_ de los datos a insertar:

https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/getting-started-python.html

https://sarahleejane.github.io/learning/python/2015/10/14/creating-an-elastic-search-index-with-python.html

El mapping se ha guardado en un fichero local "mapping_file.json", con el fin de poder modificarlo en un editor de forma más fácil (en caso de ser necesario). 

In [ ]:
file_mapping = os.path.join(os.getcwd(),'mapping_file.json')

In [ ]:
# Opening JSON file
f_map = open(file_mapping)
 
# returns JSON object as 
# a dictionary
mapping = json.load(f_map)

# Closing file
f_map.close()

In [ ]:
type(mapping)

A continuación se crea el cuerpo de la petición con el mapping del fichero.

In [ ]:
request_body = {
    "settings" : {
        "number_of_shards": 3,
        "number_of_replicas": 2,
        "index.lifecycle.name": "web-log-metrics-policy",
        "index.lifecycle.rollover_alias": "index-web-log"
    },
    'mappings': mapping
}

Como el índice del caso de uso se va a crear a diario, se introduce la fecha en el nombre del índice.

In [ ]:
test_time = datetime.now()
index_date = test_time.strftime("%Y%m%d")
print(index_date)

Finalmente, se lanza la petición de crear el índice.

In [ ]:
index_name = "index-web-log-" + index_date
if elastic_con.indices.exists(index=index_name):
    print('INFO: El indice {} ya existe.'.format(index_name))
    print('INFO: Los datos se insertarán en el indice existente.')
else:
    elastic_con.indices.create(index = index_name, body = request_body)

### Descarga de datos de log

Se obtienen la ruta donde se encuentra el log de un fichero local.

In [ ]:
file_url = os.path.join(os.getcwd(),'log_url.txt')

f_url = open(file_url,'r')
 
url_log = f_url.readline()

f_url.close()

In [ ]:
url_log

Se lee los datos de la ruta especificada y se vuelca a una variable local.

In [ ]:
try:
    jsonObj = pd.read_json(path_or_buf=url_log, lines=True, orient='records')
    fail = False
    print('OK')
except:
    fail = True
    print('KO')

Opcionalmente, se puede almacenar los registros descargados en un fichero local.

In [ ]:
data_file = os.path.join(os.getcwd(),'web_log.txt')

if fail:
    print('Falló la descarga del fichero de log.')
else:
    if os.path.isfile(data_file):
        print('El fichero ya existe en local.')
    else:
        print('Se guarda el log en local.')
        jsonObj.to_csv(data_file)

Finalmente, se incia la insercion de documentos en el índice especificado de Elasticsearch.

Para probar la política de ciclo de vida de los índices, se han creado dos varibles:
- "test" permite seleccionar un subconjunto de documentos para cargarlos en el índice de un día.
- "regs_batch" conjunto de registros que se van a insertar. NOTA: se generan dos posiciones random (una en la parte incial y otra en la parte final del fichero) a partir de las cuales se recolectan dos lotes de registros.

**IMPORTANTE: Si test = False, se cargarán todos los documentos en el índice del día.**

In [ ]:
test = False
regs_batch = 100

In [ ]:
regs_ok = 0
regs_ko = 0
list_ko = []

g1 = random.randint(1000, 6000)
g2 = random.randint(7000, 12000)

if fail:
    print('Falló la descarga del fichero de log.')
else:
    if test:
        for l in range(jsonObj.shape[0]):
            if g1 - regs_batch < l < g1 + regs_batch:
                output = bulk(elastic_con, [(jsonObj.loc[l]).to_json()], index=index_name)
                if output[0]:
                    regs_ok += 1
                else:
                    regs_ko += 1
                    list_ko.append(l)
            elif g2 - regs_batch < l < g2 + regs_batch:
                output = bulk(elastic_con, [(jsonObj.loc[l]).to_json()], index=index_name)
                if output:
                    regs_ok += 1
                else:
                    regs_ko += 1
                    list_ko.append(l)
            else:
                pass
    else:
        for l in range(jsonObj.shape[0]):
            output = bulk(elastic_con, [(jsonObj.loc[l]).to_json()], index=index_name)
            if output[0]:
                regs_ok += 1
            else:
                regs_ko += 1
                list_ko.append(l)

    print(regs_ok, regs_ko)

## CONSULTAS

En esta sección se ha intentado recopilar una consulta de cada tipo vistas durante este módulo. Se ha añadido la opción _?size=0_ para visualizar sólo el número de hits.

En el fichero "kibana_queries.txt" se incluyen una lista de queries que se ha realizado en las que se puede ver la lista de documentos. No todas se han incluido en este notebook.



### Consultas Full-text o Match Queries

#### Obtener el número de peticiones erróneas.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
    "query": { \
        "match":{ \
            "@tags": "error" \
        } \
    } \
}'

### Term queries

#### Obtener el número de peticiones de una version de mensaje específica.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
  "query": { \
    "term": { \
        "@version": "1" \
    } \
  } \
}'

### Wildcards queries

#### Obtener el número de peticiones correctas.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
  "query": { \
        "wildcard" : { \
            "response" : "2*" \
        } \
    } \
}'

#### Obtener el número de peticiones de máquinas windows.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
  "query": { \
        "wildcard" : { \
            "machine.os": "win*" \
        } \
    } \
}'

### Boolean Query

#### Obtener el número de peticiones de un cliente Mozilla 4.0.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
  "query": { \
        "bool": { \
            "must": [ \
              { "match": { "agent": "4.0" }} \
            ] \
        } \
    } \
}'

#### Obtener el número de peticiones de tamaño 0 bytes.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
    "query": { \
        "bool": { \
            "should": [ \
              { "match": { "bytes": 0 }} \
            ] \
        } \
    } \
}'

### Range query

#### Obtener el número de peticiones cuyo tamaño sea superior a 10000 bytes y darlos ordenados en orden descendente.
Nota: Para ver la ordenación hay que quitar _?size=0_ de la peticion. Además, se visualiza mejor desde Kibana.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
    "query": { \
        "range" : { \
            "bytes": { \
                "gte": "10000" \
              } \
        } \
    }, \
    "sort": [ \
      { "bytes": {"order": "desc" } } \
    ] \
}'

#### Obtener el número de peticiones entre 18/05 y el 19/05 usando el campo _@timestamp_.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
    "query": { \
        "range" : { \
            "@timestamp": { \
                "gte": "2015-05-18", \
                "lte": "2015-05-19" \
              } \
        } \
    } \
}'

#### Obtener el número de peticiones entre 19/05 y el 20/05 usando el campo _utc_time_.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
    "query": { \
        "range" : { \
            "utc_time": { \
                "gte": "2015-05-19", \
                "lte": "2015-05-20" \
              } \
        } \
    } \
}'

### Match Phrase Prefix Query

#### Obtener el número de peticiones hechas a twitter.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
    "query": { \
    "match_phrase_prefix": { \
      "headings": { \
        "query": "twitter", \
        "max_expansions": 10 \
      } \
    } \
  } \
}'

### Term aggregation (keyword)

#### Agrupar peticiones por la extensión del recurso solicitado.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
    "aggs" : { \
        "ext_agg" : { \
            "terms" : { "field" : "extension" }  \
        } \
    } \
}'

### Range aggregation (long)

#### Agrupar peticiones por el tamaño de la respuesta.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
    "aggs" : { \
        "size_agg" : { \
            "range" : { \
                "field" : "bytes", \
                "ranges" : [ \
                    { "to" : 5000 }, \
                    { "from" : 5000, "to" : 10000 }, \
                    { "from" : 10000 } \
                ] \
            } \
        } \
    } \
}'

### Date aggregation (date)

#### Agrupar peticiones anteriores y posteriores a una fecha.

In [ ]:
! curl -X GET "http://elasticsearch:9200/index-web-log-*/_search?size=0" -H 'Content-Type: application/json' -d' \
{ \
   "aggs": { \
        "data_agg": { \
            "date_range": { \
                "field": "@timestamp", \
                "format": "yyyy-MM-dd", \
                "ranges": [ \
                    { "to": "2015-05-19" },  \
                    { "from": "2015-05-19" }  \
                ] \
            } \
        } \
    } \
}'

## Conclusiones

### Opinión sobre la base de datos seleccionada como data store

La base de datos de Elasticsearch es una poderosa herramienta de búsqueda y análisis de datos. Tiene una gran capacidad para procesar grandes volúmenes de datos de manera rápida y eficiente. Está diseñado para escalar horizontalmente y gracias su integración con Kibana es posible realizar consultas complejas.

### Opinión sobre el ejercicio

El profesor a ofrecido una variedad de conjuntos de datos amplio, cada uno enfocado al uso de una de las base de datos NoSQL vistas en el módulo. No obstante, deja a consideración del alumno el poder usar una u otra base de datos, siempre y cuando justifique su uso de forma adecuada. 

El ejercicio propuesto permite al alumno definir un caso de uso a su elección, facilitándo plantear un enfoque en el que el alumno pueda mostrar lo aprendido en el módulo. Sin embargo tiene un gran inconveniente, que es la falta de conocimiento de que puede ser considerado por el profesor como suficiente.

### Qué se ha aprendido

Hasta la fecha conocía Elasticsearch desde un perfil de usuario de consulta. La realización de este módulo me ha permitido conocer el proceso de creación de indices, plantillas de indices y definir una política de gestión de ciclo de vida de los índices.

Por otro lado, se han asentado las bases para poder trabajar en un futuro con Neo4j (era mi segunda opción para la práctica) y con Cassandra.


### Carencias y mejoras

En la memoria de esta práctica se ha pretendido incluir el mayor tipo de consultas vista en el módulo de Elasticsearch. Sin embargo, debido a que este máster es muy exhaustivo en cuanto a variedad de contenidos, he dejado fuera consultas más elaboradas y vistas el notebook _14. Metric Aggregations.ipynb_. Además, como se ha explicado anteriormente, el uso de Logstash hubiese sido una opción lógica para este caso de uso, pero que se ha descartado por falta de accesibilidad a esta herramienta en el ámbito laboral.